# Our First LLMAgent — Hailstone

## Define the Hailstone Tool

In [1]:
from pydantic import BaseModel

from llm_agents_from_scratch.tools import PydanticFunctionTool


class AlgoParams(BaseModel):
    """Params for next_number."""

    x: int


def next_number(params: AlgoParams) -> int:
    """Generate the next number of the sequence."""
    if params.x % 2 == 0:
        return params.x / 2
    return 3 * params.x + 1


# convert our Python function to a BaseTool
tool = PydanticFunctionTool(next_number)

## Define our backbone LLM

In [2]:
from llm_agents_from_scratch.llms import OllamaLLM

In [3]:
llm = OllamaLLM(model="qwen2.5:72b")

## Define the LLMAgent

In [4]:
from llm_agents_from_scratch import LLMAgent

In [5]:
llm_agent = LLMAgent(
    llm=llm,
    tools=[tool],
)

## Define the Hailstone Task

In [6]:
from llm_agents_from_scratch.data_structures import Task

In [7]:
task_instruction = """
You are given a tool, `next_number`, that generates the following number in the sequence given
the latest number.

Start with the number x=15.

Step-by-Step Process:

Make ONE tool call only - Call the `next_number` tool with the current number
Never fabricate or assume tool results - Only work with the actual returned value
Evaluate the result - Once you receive the real tool output, determine if you should continue or exit
Exit only if the tool outputs 1
If continuing - Make another single tool call in your next response

Critical Rules:

NEVER make multiple tool calls in one response
NEVER invent or simulate tool call results
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
Each iteration = exactly one tool call + evaluation of the real result

FINAL RESULT: When you receive the number 1, provide the complete sequence 
you observed from start to finish (including the starting number x and 
ending number 1).
"""

In [8]:
task = Task(
    instruction=task_instruction,
)

In [9]:
LOGGING_ENABLED = True

In [10]:
import logging

from llm_agents_from_scratch.logger import enable_console_logging

if LOGGING_ENABLED:
    enable_console_logging(logging.INFO)

In [11]:
handler = llm_agent.run(task)

INFO (llm_agents_fs.LLMAgent) :      🚀 Starting task: 
You are given a tool, `next_number`, that generates the following number in the sequence given
the latest number.

Start with the number x=15.

Step-by-Step Process:

Make ONE tool call only - Call the `next_number` tool with the current number
Never fabricate or assume tool results - Only work with the actual returned value
Evaluate the result - Once you receive the real tool output, determine if you should continue or exit
Exit only if the tool outputs 1
If continuing - Make another single tool call in your next response

Critical Rules:

NEVER make multiple tool calls in one response
NEVER invent or simulate tool call results
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
Each iteration = exactly one tool call + evaluation of the real result

FINAL RESULT: When you receive the number 1, provide the complete sequence 
you observed from start to finish (including the starting number x and 
endi

In [13]:
handler.done()

True

# See the TaskResult

In [14]:
result = handler.result()

In [15]:
result

TaskResult(task_id='1ea7a080-1756-4527-8a6d-01e9b204ed83', content='Thank you for confirming! The final sequence is:\n\n- Start: 15\n- 46\n- 23\n- 70\n- 35\n- 106\n- 53\n- 160\n- 80\n- 40\n- 20\n- 10\n- 5\n- 16\n- 8\n- 4\n- 2\n- **End: 1**\n\nThe sequence is complete and accurate. If you have any more tasks or questions, feel free to let me know!')

### See the Rollout

In [16]:
print(handler.rollout)

💬 assistant: The current instruction is '
You are given a tool, `next_number`, that generates the following number in the sequence given
the latest number.

Start with the number x=15.

Step-by-Step Process:

Make ONE tool call only - Call the `next_number` tool with the current number
Never fabricate or assume tool results - Only work with the actual returned value
Evaluate the result - Once you receive the real tool output, determine if you should continue or exit
Exit only if the tool outputs 1
If continuing - Make another single tool call in your next response

Critical Rules:

NEVER make multiple tool calls in one response
NEVER invent or simulate tool call results
STOP and WAIT - ALWAYS wait for the actual tool response before deciding next steps
Each iteration = exactly one tool call + evaluation of the real result

FINAL RESULT: When you receive the number 1, provide the complete sequence 
you observed from start to finish (including the starting number x and 
ending number 1).

In [ ]:
Agent runs Tasks by performing cycles of (get_next_step, run_step). If only get_next_step is run, and not run_step i.e., because TaskResult was returned in get_next_step, then this is a still a cycle